### Chap4 Recursion

In [2]:
from typing import List, TypeVar, Tuple, Any
from functools import lru_cache
Num = TypeVar('Num', int, float)

#### Reinforcement

##### R-4.1 
Describe a recursive algorithm for finding the maximum element in a sequence, S, of n elements. What is your running time and space usage?

In [2]:
def max_recursion(nums: List[Num], n: int) -> Num:
    if n == 1:
        return nums[0]
    return max(nums[n-1], n-1)

In [4]:
max_recursion([1, 3, 2, 6, 10], 5)

10

时间复杂度为$O(n)$, 因为没有用到额外的空间，所以空间复杂度为$O(1)$.

##### R-4.6
Describe a recursive function for computing the n th Harmonic number, $H_n = \sum_{i=1}^{n} \frac{1}{n}$.

In [5]:
def harmonic_recursion_1(n: int) -> Num:
    if n == 1:
        return n
    return 1/n + harmonic_recursion_1(n-1)

# python doesn't support tail-call optimization
def harmonic_recursion_2(n: int, acc = 0) -> Num:
    if n == 0:
        return acc
    return harmonic_recursion_2(n-1, acc + 1/n)

@lru_cache(maxsize=None)
def harmonic_recursion_3(n: int) -> Num:
    if n == 1:
        return n
    return 1/n + harmonic_recursion_1(n-1)

第一个是普通版本；第二个是采用了尾递归的写法（但是Python并没有进行尾递归优化）；第三个是采用了内建的`lru_cache`。Benchmerking结果如下：
```
In [49]: %timeit harmonic_recursion_1(50)                                                                                                                      
9.01 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

In [50]: %timeit harmonic_recursion_2(50)                                                                                                                      
11.3 µs ± 62.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

In [51]: %timeit harmonic_recursion_3(50)                                                                                                                      
111 ns ± 0.657 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
```

前两个差别不大，这里*并没有*搞清楚为什么第三个写法会比较快。因为按照文档所说，这里的cache,应该仅仅在应用DP算法的时候可以作为备查表，但是这里不同的参数值仅仅是调用了一次，所以理应不会有什么优化才对...

##### R-4.7 
Describe a recursive function for converting a string of digits into the in-
teger it represents. For example, '13531' represents the integer 13, 531.

In [6]:
def char_to_int(char: str) -> int:
    """
    convert the single number with str type to number type
    """
    return ord(char) - ord('0')

def str_to_int(string: str, n: int) -> int:
    if n == 1:
        return char_to_int(string[0])
    return char_to_int(string[n-1]) * (10 ** (n-1)) \
            + str_to_int(string, n-1)

In [8]:
str_to_int('13531', 5)

13531

##### R-4.8 
Isabel has an interesting way of summing up the values in a sequence A of
n integers, where n is a power of two. She creates a new sequence B of half
the size of A and sets B[i] = A[2i] + A[2i + 1], for i = 0, 1, . . . , (n/2) − 1. If
B has size 1, then she outputs B[0]. Otherwise, she replaces A with B, and
repeats the process. What is the running time of her algorithm?

solution:

首先，不妨设$n = 2^m, m\in \mathbb{N^+}$.那么，必须经过$\log n = m$次操作才能完成。将每次的运算次数加总，
$$\frac{n}{2^1} + \frac{n}{2^2} + \cdots + \frac{n}{2^m} = n \cdot [1-(\frac{1}{2})^m]
= n \cdot (1 - \frac{1}{n}) = n - 1 => O(n)$$
其中，
$$(\frac{1}{2})^m = (\frac{1}{2})^{\log n} = 2^{- \log n} = \frac{1}{n}$$

#### Creativity

##### C-4.9
Write a short recursive Python function that finds the minimum and max-
imum values in a sequence without using any loops.

In [3]:
def my_min(n1: Num, n2: Num) -> Num:
    if n1 >= n2:
        return n2
    else:
        return n1

def my_max(n1: Num, n2: Num) -> Num:
    if n1 >= n2:
        return n1
    else:
        return n2

def min_max_num(nums: List[Num], n: int) -> Tuple[Num, Num]:
    if n == 1:
        return (nums[0], nums[0])
    return (
            my_min(nums[n-1], min_max_num(nums, n-1)[0]),
            my_max(nums[n-1], min_max_num(nums, n-1)[1]),
            )

注意这里，我们默认是不适用Python的内建求最值的函数的，不然也太无趣了>.<

In [4]:
min_max_num([1, 3, 2, 4, 7], 5)

(1, 7)

##### C-4.10 
Describe a recursive algorithm to compute the integer part of the base-two
logarithm of n using only addition and integer division.

In [6]:
def get_log_int(n: Num) -> int:
    assert n > 0
    if n < 2:
        return 0
    return 1 + get_log_int(n // 2)

In [7]:
get_log_int(16), get_log_int(15)

(4, 3)

##### C-4.11
Describe an efficient recursive function for solving the element unique-
ness problem, which runs in time that is at most $O(n^2)$ in the worst case
without using sorting.

In [8]:
def one_diff_all(element:Any, seq: List[Any], length: int) -> bool:
    for i in range(length):
        if element == seq[i]:
            return False
    return True

def is_unique(seq: List[Any], n: int) -> bool:
    if n == 1:
        return True
    return is_unique(seq, n-1) and one_diff_all(seq[n-1], seq, n-1)

In [9]:
is_unique([1, 2, 3, 4], 4), is_unique([1, 2, 3, 3, 4], 5)

(True, False)

其实这里的思路还是很简单的，不过这里我们要注意不能创建或者复制太多的列表元素，其会占用很多空间，且复制会加大时间成本。此外，我们在Haskell可以更加清晰地看出算法的逻辑：
```Haskell
allDifferent :: (Eq a) => [a] -> Bool
allDifferent list = case list of
    []      -> True
    (x:xs)  -> x `notElem` xs && allDifferent xs
```

##### C-4.12
Give a recursive algorithm to compute the product of two positive integers,
m and n, using only addition and subtraction.

In [10]:
def multiply(m: int, n: int) -> int:
    if n == 1:
        return m
    return m + multiply(m, n-1)

In [11]:
multiply(3, 4)

12

##### C-4.15
Write a recursive function that will output all the subsets of a set of n
elements (without repeating any subsets).

In [12]:
def get_subsets(s: List[Any], n: int) -> List[Any]:
    if n == 0:
        return [[]]
    return [[s[n-1]] + i for i in get_subsets(s, n-1)] + get_subsets(s, n-1)

In [15]:
get_subsets(['a', 'b', 'c'], 3)

[['c', 'b', 'a'], ['c', 'b'], ['c', 'a'], ['c'], ['b', 'a'], ['b'], ['a'], []]

##### C-4.16
Write a short recursive Python function that takes a character string s and
outputs its reverse. For example, the reverse of 'pots&pans' would be
'snap&stop'.

In [16]:
def reverse_str(s: str, n: int) -> str:
    if n == 1:
        return s[0]
    return s[n-1] + reverse_str(s, n-1)

In [17]:
reverse_str('pots&pans', 9)

'snap&stop'

##### C-4.17
Write a short recursive Python function that determines if a string s is a
palindrome, that is, it is equal to its reverse. For example, 'racecar' and
'gohangasalamiimalasagnahog' are palindromes.

In [18]:
def is_palindrome(s: str) -> bool:
    def judge(s: str, start: int, end: int) -> bool:
        n = end - start + 1
        if n <= 1:
            return True
        return (s[start] == s[end]) and judge(s, start+1, end-1)
    return judge(s, 0, len(s)-1)

In [20]:
is_palindrome('racecar'), is_palindrome('racecarss')

(True, False)

##### C-4.18
Use recursion to write a Python function for determining if a string s has
more vowels than consonants.

In [21]:
def count_vowels(s: str, n: int) -> int:
    vowels = 'aeiouAEIOU'
    if n == 0:
        return 0
    if s[n-1] in vowels:
        return 1 + count_vowels(s, n-1)
    else:
        return count_vowels(s, n-1)

def is_more_vowel(s: str) -> bool:
    num_vowel = count_vowels(s, len(s))
    num_consonants = len(s) - num_vowel
    if num_vowel > num_consonants:
        return True
    else:
        return False

In [22]:
is_more_vowel('hello')

False

##### C-4.19
Write a short recursive Python function that rearranges a sequence of integer values so that all the even values appear before all the odd values.

In [23]:
def even_before_odd(nums: List[int]) -> List[int]:
    if not nums:
        return []
    if nums[0] % 2 == 0:
        return [nums[0]] + even_before_odd(nums[1:])
    else:
        return even_before_odd(nums[1:]) + [nums[0]]

In [24]:
even_before_odd([1, 2, 3, 4])

[2, 4, 3, 1]

##### C-4.20
Given an unsorted sequence, S, of integers and an integer k, describe a
recursive algorithm for rearranging the elements in S so that all elements
less than or equal to k come before any elements larger than k. What is
the running time of your algorithm on a sequence of n values?

In [25]:
def seperate_by_k(nums: List[int], k: int) -> List[int]:
    if not nums:
        return []
    if nums[0] <= k:
        return [nums[0]] + seperate_by_k(nums[1:], k)
    else:
        return seperate_by_k(nums[1:], k) + [nums[0]]

In [26]:
seperate_by_k([1, 2, 3, 7, 5, 3], 4)

[1, 2, 3, 3, 5, 7]

##### C-4.21
Suppose you are given an n-element sequence, S, containing distinct in-
tegers that are listed in increasing order. Given a number k, describe a
recursive algorithm to find two integers in S that sum to k, if such a pair
exists. What is the running time of your algorithm?

>hint:The beginning and the end of a range of indices in S can be used
as arguments to your recursive function.

In [27]:
def sum_to_k(nums: List[int], k: int, start: int, end: int) -> List[int]:
    assert len(nums) > 2
    if start == end:
        return []
    if nums[start] + nums[end] == k:
        return [nums[start], nums[end]]
    elif nums[start] + nums[end] < k:
        return sum_to_k(nums, k, start+1, end)
    else:
        return sum_to_k(nums, k, start, end-1)

In [28]:
sum_to_k([1, 2, 3, 4], 5, 0, 3)

[1, 4]

##### C-4.22
Develop a nonrecursive implementation of the version of power from
Code Fragment 4.12 that uses repeated squaring.

参考[stackoverflow](https://stackoverflow.com/questions/23079443/c-x-to-the-power-n-using-repeated-squaring-without-recursive-function)

In [29]:
def power(x: int, n: int):
    result = 1
    while n > 0:
        while n & 1 == 0:
            n //= 2
            result *= result
        result *= x
        n -= 1
    return result

In [30]:
power(2, 5)

32